In [1]:
import pandas as pd
import numpy as np

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [13]:
import requests

result = requests.get(URL).text

In [14]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(result,'lxml')

In [123]:
soup
Postcode = []
Borough = []
Neighborhood = []

# Find tables
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    # Collect table entries
    data = items.find_all(['th','td'])
    # If no info presented for 'Borough' -> skip
    if data[1].text == 'Not assigned':
        continue
    # Process entries with the same PostalCode
    idx = -1
    try:
        idx = Postcode.index(data[0].text)
    except:
        idx = -1
    if idx != -1:
        Neighborhood[idx] += ', ' + data[2].text[:-1]
        continue
        
    Postcode.append(data[0].text)
    Borough.append(data[1].text)
    # Set neighborhood value to borough value if neighborhood value is not defined
    if data[2].text[:-1] == 'Not assigned':
        Neighborhood.append(data[1].text)
        continue
        
    Neighborhood.append(data[2].text[:-1])

In [124]:
df = pd.DataFrame([Postcode, Borough, Neighborhood]).T

In [130]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [131]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [132]:
df.shape

(103, 3)

In [156]:
df_loc = pd.read_csv('location.csv')

In [162]:
df_loc_cleaned = df_loc.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1).rename(columns={'Postal Code' : 'PostalCode'})

In [163]:
df_loc_cleaned.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [165]:
df_merged = pd.merge(df, df_loc_cleaned, how='inner', on='PostalCode')

In [166]:
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
